In [1]:
!pip install pyautogen[ollama]

  Using cached pyautogen-0.10.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached pyautogen-0.10.0-py3-none-any.whl (3.0 kB)
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)

  Attempting uninstall: protobuf

    Found existing installation: protobuf 6.33.2

    Uninstalling protobuf-6.33.2:

      Successfully uninstalled protobuf-6.33.2

   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   -----------------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.76.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.5 which is incompatible.


In [2]:
!pip install ollama fix-busted-json

  Using cached ollama-0.6.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached fix_busted_json-0.0.18-py3-none-any.whl.metadata (4.6 kB)
Using cached ollama-0.6.1-py3-none-any.whl (14 kB)
Using cached fix_busted_json-0.0.18-py3-none-any.whl (7.4 kB)

   ---------------------------------------- 2/2 [ollama]



# Setting base for Retrive Chat

In [1]:
import json
import os

import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST.json")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-35-turbo', 'gpt-4-vision-preview', 'dalle', 'gemini-pro', 'gemini-1.5-pro-001', 'gemini-1.5-pro', 'gemini-pro-vision']


In [7]:
print("Accepted file formats for `docs_path`:")

Accepted file formats for `docs_path`:


In [8]:
config_list = [
    {
        # Let's choose the Meta's Llama 3.1 model (model names must match Ollama exactly)
        "model": "llama3",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        #"client_host": "http://192.168.0.1:11434",
    }
]

In [9]:
import agentops
agentops.init("<Enter your key>")

In [11]:
# 1. create an AssistantAgent instance named "assistant"
from autogen import AssistantAgent

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)


🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
C:\Users\Sai Vennela Jagadesh\anaconda3\envs\sai\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
🖇 AgentOps: Google Generative AI instrumentation enabled
C:\Users\Sai Vennela Jagadesh\anaconda3\envs\sai\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.cloud.aiplatform_v1beta1 once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least

In [12]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Extract text from local PDF files
pdf_files = ["book.pdf"]
#docs_content = [extract_text_from_pdf(file) for file in pdf_files]

from autogen.retrieve_utils import TEXT_FORMATS,extract_text_from_pdf
docs_content= extract_text_from_pdf("book.pdf")

In [13]:
# Define the RetrieveUserProxyAgent instance
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": ["autogen-docs/book.pdf"],
        "docs_content": docs_content,  # Provide the extracted content directly
        "chunk_token_size": 200,
        "model": config_list[0]["model"],
        "vector_db": "chroma",
        "overwrite": True,
        "get_or_create": True,
    },
    code_execution_config=False,
)

# Process Starts for Scenario 1 : Generate code based off docstrings w/o human feedback

In [14]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "summarize the content and use python code to any calculation given in the book"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)
#agentops.end_session("Success")

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	The 22 ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2025-12-24 18:39:38,721 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 215 chunks.
2025-12-24 18:39:38,732 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Model llama3 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['fe1105cf', '8bf7f8ee', '839a2e04', 'd11d708f', '150c596a', 'fbda2b4c', '59e93e46', '1613971f', '25edaaee', '7091167f', 'c1c95a95', '43a28a91', '16a1e2eb', 'e7d73411', 'a5e9b671', '05dac356', '1097b211', 'd41e75dd', '52a863ba', '10e789b5']]
Adding content of doc fe1105cf to context.


Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 8bf7f8ee to context.


...

Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 1097b211 to context.


Model llama3 not found. Using cl100k_base encoding.


ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: summarize the content and use python code to any calculation given in the book

Context is: domestic light beer, maybe there’ s a market for an
imported light beer.” The result was Amstel Light,
which became the largest-selling imported light beer.
If you didn’t get into the prospect’ s mind first, don’t
give up hope. Find a new category you can be first in.
It’ s not as difficult as you might think.
After IBM became a big success in computers,
everybody and his brother jumped into the field.
11
22 Laws of Marketing  10/31/02  12:23 PM  Page 11Burroughs, Control Data, General Electric,
Honeywell, NCR, RCA, Sperry. Snow White and the
seven dwarfs, the

ResponseError: llama runner process has terminated: CUDA error (status code: 500)

In [15]:
#agentops.end_session("Success")

# Process Starts for Scenario 2 : Answer a question based off docstrings w/o human feedback

In [16]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "You are a creative head . do many brainstorms"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)

Model llama3 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['1097b211', '7e1ac0cd', '0e08a976', '81d8712b', 'be843ac3', 'a5e9b671', 'b2f1dbea', 'ba9a27e8', '21ec236c', '8126c05a', 'cf32340a', '7e3514a1', '89fd1dac', '5b19ade7', '10e789b5', '0e403d83', '71a3bd57', '1e7668a9', 'a1f246ff', '4cf44936']]
Adding content of doc 1097b211 to context.


Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 7e1ac0cd to context.


Model llama3 not found. Using cl100k_base encoding.


...

Adding content of doc 10e789b5 to context.


Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 0e403d83 to context.


Model llama3 not found. Using cl100k_base encoding.


ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: You are a creative head . do many brainstorms

Context is: ing is a long-term trend. 123
THE LA W OF ACCELERATION
22 Laws of Marketing  10/31/02  12:23 PM  Page 12322
The Law of 
Resources
Without adequate funding
an idea won’t get off the ground.
124
22 Laws of Marketing  10/31/02  12:23 PM  Page 124If you have a good idea and you’ve picked up this
book with the thought in mind that all you need is a lit-
tle marketing help, this chapter will throw cold water
on that thought.
Even the best idea in the world won’t go very far
without the money to get it off the ground. Inventors,
entrepreneurs, and assorted idea generators seem to
think that all thei

# Process Starts for Scenario 3 : Generate code based off docstrings w/ human feedback

In [17]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "Based on this book give Marketing idea for AI and Datascience trainig institute. Give 5 ideas. if any coding requires use python and save it as .py"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=code_problem)

Model llama3 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['1097b211', '81d8712b', '89fd1dac', '0e08a976', '71a3bd57', '150c596a', 'be843ac3', 'ba9a27e8', '8bf7f8ee', '8a2a101c', 'd0e14ddf', 'b210f301', 'c1b21496', 'd5a791b1', 'a5e9b671', 'fc6e8864', '0e403d83', 'ef80c94d', '08e00728', 'ff30ed4d']]
Adding content of doc 1097b211 to context.


Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 81d8712b to context.


Model llama3 not found. Using cl100k_base encoding.


...

Adding content of doc ef80c94d to context.


Model llama3 not found. Using cl100k_base encoding.


ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Based on this book give Marketing idea for AI and Datascience trainig institute. Give 5 ideas. if any coding requires use python and save it as .py

Context is: ing is a long-term trend. 123
THE LA W OF ACCELERATION
22 Laws of Marketing  10/31/02  12:23 PM  Page 12322
The Law of 
Resources
Without adequate funding
an idea won’t get off the ground.
124
22 Laws of Marketing  10/31/02  12:23 PM  Page 124If you have a good idea and you’ve picked up this
book with the thought in mind that all you need is a lit-
tle marketing help, this chapter will throw cold water
on that thought.
Even the best idea in the world won’t go very far
without the money to get

Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (951e1366-3271-4d59-9b6b-c9f672b96080): User requested to end the conversation


# Process Starts for Scenario 4: Answer a question based off docstrings w/ human feedback

In [18]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "Based on this book give Marketing idea for AI and Datascience trainig institute. Give 5 ideas. "
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=qa_problem
)  # type "exit" to exit the conversation

Model llama3 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['1097b211', '81d8712b', '71a3bd57', '89fd1dac', 'be843ac3', '150c596a', 'b210f301', '8a2a101c', 'ba9a27e8', '0e08a976', '0e403d83', '788184ee', '039e8163', 'a1f246ff', 'd5a791b1', '08e00728', '083871f5', 'c1b21496', '0c4927c1', '8cc896cd']]
Adding content of doc 1097b211 to context.


Model llama3 not found. Using cl100k_base encoding.


Adding content of doc 81d8712b to context.


Model llama3 not found. Using cl100k_base encoding.


...

Adding content of doc c1b21496 to context.


Model llama3 not found. Using cl100k_base encoding.


ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Based on this book give Marketing idea for AI and Datascience trainig institute. Give 5 ideas. 

Context is: ing is a long-term trend. 123
THE LA W OF ACCELERATION
22 Laws of Marketing  10/31/02  12:23 PM  Page 12322
The Law of 
Resources
Without adequate funding
an idea won’t get off the ground.
124
22 Laws of Marketing  10/31/02  12:23 PM  Page 124If you have a good idea and you’ve picked up this
book with the thought in mind that all you need is a lit-
tle marketing help, this chapter will throw cold water
on that thought.
Even the best idea in the world won’t go very far
without the money to get it off the ground. Inventors,
entrepreneurs, and as

Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (3418dba4-35c6-4f3f-91c6-9ed6d8473a60): User requested to end the conversation
